In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf


def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [2]:
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index

In [35]:
# Get Stock Market Data: SHY (1-3 Year Treasury bond ETF)
import yfinance as yf
stock_info = yf.download("SHY", start="1993-02-01", end="2021-02-20")
stock_info.index = pd.to_datetime(stock_info.index)
stock_info

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-07-30,81.120003,81.129997,81.000000,81.010002,58.842762,95300
2002-07-31,81.169998,81.309998,81.150002,81.260002,59.024364,38800
2002-08-01,81.330002,81.449997,81.330002,81.419998,59.140572,86800
2002-08-02,81.459999,81.639999,81.400002,81.580002,59.256790,83400
2002-08-05,81.629997,81.730003,81.570000,81.709999,59.351196,93100
2002-08-06,81.610001,81.610001,81.449997,81.510002,59.205971,58200
2002-08-07,81.589996,81.690002,81.540001,81.629997,59.293156,127000
2002-08-08,81.610001,81.620003,81.470001,81.510002,59.205971,111800
2002-08-09,81.480003,81.559998,81.459999,81.529999,59.220440,50900


In [36]:
df1 = df[df['Year']> "2002-07-30"]

In [59]:
# stock market volatibility within 10 days after the Fed meetings, window = 10 performs the best
import numpy as np
price_changes = []
price_change_percentages = []
labels = []
window = 10
# workaround for rolling widow
for index, row in df1.iterrows():
    minutes_date = row['Year']
    try:
        start = stock_info.index.get_loc(minutes_date)
    except:
        start = stock_info.index.get_loc(minutes_date + pd.DateOffset(days=1))
    stock_closing_sum = stock_info.iloc[start+window]['Close']
    start_price = stock_info['Close'].iloc[start]
    price_change = stock_closing_sum - start_price
    price_change_percent = (stock_closing_sum - start_price) / start_price
    price_changes.append(price_change)
    price_change_percentages.append(price_change_percent)
    

df1['price_change'] = price_changes
df1['price_change_percent'] = price_change_percentages

#label =1 means high volatility, label = 1 means low volatility. High volatility is identified if VIX price change percent is far from its median (greater than 75% or smaller than 25%)
df1['label'] = df1['price_change_percent'].apply(lambda x: 1 if x> df1["price_change_percent"].quantile(0.75) or x < df1["price_change_percent"].quantile(0.25) else 0)

C:\Users\wanyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wanyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wanyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [60]:
X = df1
X = np.array(X.drop(['label','Statement','Year','positive','negative','price_change','price_change_percent'],1))
y = np.array(df1['label'])

In [61]:
#Split(80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.4, random_state = 0)

In [62]:
# Create and train model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [63]:
predictions = model.predict(x_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.57      0.59      0.58        27
           1       0.61      0.59      0.60        29

    accuracy                           0.59        56
   macro avg       0.59      0.59      0.59        56
weighted avg       0.59      0.59      0.59        56



In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [65]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
m = randomclassifier.fit(x_train,y_train)
predictions_2 = m.predict(x_test)
score = accuracy_score(y_test,predictions)
report = classification_report(y_test,predictions)
print(score)
print(report)

0.5892857142857143
              precision    recall  f1-score   support

           0       0.57      0.59      0.58        27
           1       0.61      0.59      0.60        29

    accuracy                           0.59        56
   macro avg       0.59      0.59      0.59        56
weighted avg       0.59      0.59      0.59        56

